In [20]:
import copy
# Import required modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os.path
import pickle

import argparse
import torch

import random
import numpy.random
import torch.random

from datetime import datetime

import warnings
warnings.filterwarnings(action='ignore')

In [26]:
# Define variables
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

# 2023-05-02
# For Using GPU (CUDA)
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2023-05-02
# For using tensorboard
args.istensorboard = False

args.total_episode = 100
args.learning_rate = 0.1 # Learning rate Alpha
args.boltzmann_tau = 5.0 # Initial temperature parameter at Boltzmann policy, Tau
args.total_reward = 0
args.epsilon = 0.1

args.max_iteration = 50 # Maximum iteration num. of algorithm, MAX_STEPS
args.replay_batch = 100 # Replay batch size, B
args.nn_update_num = 20 # CNN update number, U: [(1) Constant num. of iteration], (2) Lower limit of loss function value
args.batch_size = 20 # Batch size, N
args.replay_memory = [] # (1) Using list class, (2) Replay memory class by SLM Lab (page. 105~108)

args.gridnum_x = 15
args.gridnum_y = 15
args.gridsize_x = 120 # ft
args.gridsize_y = 120 # ft

args.time_step = 120 # days
args.total_production_time = 600 # days

# 2023-05-02: For reproduction
args.random_seed = 202022673
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

# State: Pressure distribution, Oil saturation

# Action: Well placement (Coordinate of well location)

# Environment: Reservoir simulator

# Reward: NPV at each time segment

args.discount_rate = 0.1 # Used for calculation of NPV
args.discount_factor = 1 # Used for Q-value update

args.simulation_directory = "H:\\Lab_Meeting\\Simulation_Model\\2D_15-by-15_DQN\\2D_15-by-15_DQN_Filetest"

In [27]:
# with open("H:\\Lab_Meeting\\Simulation_Model\\2D_15-by-15_DQN\\2D_15-by-15_DQN_Filetest\\2D_JY_ECLRUN_1.PRT", 'r') as file_read:
#     line = file_read.readline()
#     print(line)
#     while not line.startswith(f"  PRESSURE AT   {120*1}"):
#         line = file_read.readline()
#     for i in range(1,10+1):
#         line = file_read.readline()
#     print(line)
#
#     line_temp = []
#     lines_converted = []
#     for i in range(1, 15+1):
#         # line_temp = [element.strip() for element in line.split()]
#         lines_converted.append([element.strip() for element in line.split()][3::])
#         line = file_read.readline()
#     print(lines_converted)
# print(f"  PRESSURE AT   {120*1}")

################################# Reading Dynamic Data #################################
# idx: simulation file index
# tstep_idx: simulation time step index
# filename: simulation file name
# dynamic_type: dynamic data type to collect ('PRESSURE' or 'SOIL')
def _read_ecl_prt(args, idx: int, tstep_idx: int, filename: str, dynamic_type: str) -> list:
    # Check if dynamic type input is (1) 'PRESSURE', (2) 'SOIL'
    if not dynamic_type in ['PRESSURE', 'SOIL']:
        print("Assign correct dynamic data output type!: 'PRESSURE', 'SOIL'")
        return -1

    # File IO
    # 1. Open .PRT file
    with open(args.simulation_directory+'\\'+filename+'_'+str(idx)+'.PRT') as file_read:
        line = file_read.readline()
        if dynamic_type == 'PRESSURE':
            # 2. Fine the location of dynamic data (PRESSURE case)
            while not line.startswith(f"  {dynamic_type} AT   {args.time_step * tstep_idx}"):
                line = file_read.readline()
            # 3. Dynamic data is located at 10th line below the line ["  {dynamic_type} AT   {args.time_step * tstep_idx}"]
            for i in range(1,10+1):
                line = file_read.readline()
            # 4. Collect dynamic data
            lines_converted = []
            for i in range(1, args.gridnum_y+1):
                lines_converted.append([element.strip() for element in line.split()][3::])
                line = file_read.readline()
        elif dynamic_type == 'SOIL':
            # 2. Fine the location of dynamic data (SOIL case)
            while not line.startswith(f"  {dynamic_type}     AT   {args.time_step * tstep_idx}"):
                line = file_read.readline()
            # 3. Dynamic data is located at 10th line below the line ["  {dynamic_type} AT   {args.time_step * tstep_idx}"]
            for i in range(1,10+1):
                line = file_read.readline()
            # 4. Collect dynamic data
            lines_converted = []
            for i in range(1, args.gridnum_y+1):
                lines_converted.append([element.strip() for element in line.split()][3::])
                line = file_read.readline()

    # 5. Post-processing (String replacement from (1) '*' to '.', (2) String to Float)
    for i in range(len(lines_converted)):
        for j in range(len(lines_converted[i])):
            lines_converted[i][j] = float(lines_converted[i][j].replace('*', '.'))

    return lines_converted


test = _read_ecl_prt(args, 1, 2, '2D_JY_ECLRUN', 'PRESSURE')
print(test)

# print(test)

[[2150.96, 2097.01, 2029.83, 1890.89, 1706.59, 1814.43, 1956.97, 2125.1, 2131.02, 1965.12, 1670.66, 1557.06, 1534.02, 1527.14, 1526.33], [2082.75, 2057.76, 2008.36, 1911.3, 1823.34, 1829.44, 1861.85, 1891.1, 1844.18, 1707.45, 1622.87, 1553.2, 1530.66, 1525.75, 1524.83], [2034.26, 2013.03, 1962.96, 1897.48, 1840.44, 1794.8, 1765.94, 1755.4, 1688.11, 1623.46, 1575.18, 1542.8, 1526.04, 1522.34, 1521.65], [1978.57, 1963.81, 1902.62, 1857.74, 1774.76, 1708.74, 1679.76, 1642.79, 1588.35, 1554.03, 1539.91, 1528.95, 1520.24, 1518.68, 1518.73], [1919.54, 1882.45, 1824.01, 1780.28, 1671.76, 1639.07, 1602.81, 1571.47, 1544.92, 1535.05, 1528.63, 1521.42, 1515.54, 1516.11, 1516.71], [1844.56, 1794.12, 1772.03, 1689.65, 1606.7, 1580.54, 1556.19, 1542.33, 1532.93, 1528.15, 1524.18, 1517.89, 1509.33, 1513.7, 1515.15], [1722.66, 1686.67, 1656.78, 1596.26, 1572.25, 1552.94, 1541.08, 1529.04, 1526.31, 1524.98, 1523.33, 1520.07, 1515.29, 1515.31, 1515.95], [1592.87, 1586.24, 1572.76, 1563.68, 1551.97, 154

In [28]:
for i in range(0,15):
    print(len(test[i]))

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15


In [34]:
def _read_ecl_rsm(args, idx: int, filename: str, data_type: str) -> list:
    # Check if data type input is (1) 'FOPT', (2) 'FWPT', (3) 'FWIT'
    if not data_type in ['FOPT', 'FWPT', 'FWIT']:
        print("Assign correct output data type!: 'FOPT', 'FWPT', 'FWIT'")
        return -1

    # File IO
    # 1. Open .RSM file
    with open(args.simulation_directory+'\\'+filename+'_'+str(idx)+'.RSM') as file_read:
        line = file_read.readline()
        # 2. Fine the location of dynamic data (PRESSURE case)
        while not line.startswith(f" TIME"):
            line = file_read.readline()
        # 3. RSM data is located at 6th line below the line [" TIME"]
        for i in range(1,5+1+1):
            line = file_read.readline()
        # 4. Collect production or injection data
        lines_converted = []
        if data_type == 'FOPT':
            for i in range(1, round(args.total_production_time/args.time_step)+1):
                lines_converted.append([element.strip() for element in line.split()][2])
                line = file_read.readline()
        elif data_type == 'FWPT':
            for i in range(1, round(args.total_production_time/args.time_step)+1):
                lines_converted.append([element.strip() for element in line.split()][3])
                line = file_read.readline()
        elif data_type == 'FWIT':
            for i in range(1, round(args.total_production_time/args.time_step)+1):
                lines_converted.append([element.strip() for element in line.split()][4])
                line = file_read.readline()

    # 5. Post-processing (String replacement from (1) '*' to '.', (2) String to Float, only for 2D)
    for i in range(len(lines_converted)):
        lines_converted[i] = float(lines_converted[i].replace('*', '.'))

    return lines_converted

test = _read_ecl_rsm(args, 1, '2D_JY_ECLRUN', 'FWIT')
print(test)

[0.0, 0.0, 0.0, 0.0, 0.0]


In [40]:
import numpy as np
from copy import deepcopy
def Boltzmann_policy(args, Q_value: list, well_placement: list) -> list:
    exp_tau = deepcopy(Q_value)
    probability = deepcopy(Q_value)

    # Preventing overflow error
    max_Q_value = np.array(Q_value).flatten().max()

    # Get exponential of all elements in Q_value
    for i in range(0, len(Q_value)):
        for j in range(0, len(Q_value[i])):
            exp_tau[i][j] = np.exp((exp_tau[i][j]-max_Q_value)/args.boltzmann_tau)

    # Calculating probability map
    for i in range(0, len(Q_value)):
        for j in range(0, len(Q_value[i])):
            probability[i][j] = exp_tau[i][j] / np.concatenate(np.array(exp_tau)).sum()

    # Masking well placement map
    probability = [[0 if well_placement[i][j] != 0 else probability[i][j] for j in range(len(Q_value[i]))] for i in range(len(Q_value))]
    probability = [[(probability[i][j]/np.concatenate(np.array(probability)).sum()) for j in range(len(Q_value[i]))] for i in range(len(Q_value))]

    return probability

test = Boltzmann_policy(args, [[1,2,3],[4,5,6],[7,8,9]], [[0,0,0],[1,0,1],[0,0,1]])
test

[[0.04384519111888324, 0.053552637364663785, 0.06540933898395172],
 [0.07989114704443215, 0.09757926735264912, 0.11918358628377426],
 [0.14557116101442247, 0.17780101757159375, 0.21716665326562945]]

In [13]:
test = ([[11,2,3],[4,5,6],[7,8,9]])/2

TypeError: unsupported operand type(s) for /: 'list' and 'int'